In [8]:
# !git clone https://github.com/TomerRonen34/mixed-resolution-vit.git
# %mv mixed-resolution-vit/* ./
# %rm -rf mixed-resolution-vit

In [1]:
import torch

from tqdm import tqdm
from time import time

from vit import *
from utils import *
from dataloader import *
from decatt import DecattLoss

torch.manual_seed(0)

In [2]:
device_ids = [i for i in range(torch.cuda.device_count())]
print(device_ids)

[0]


# CIFAR10

In [3]:
image_size = 256
min_patch_size = 16
max_patch_size = 64
quadtree_num_patches = 64
in_channels = 3
mlp_dim = 512
num_classes = 10
num_layers = 12
dropout = 0.1
batch_size = 256

lr = 1e-3
weight_decay = 1e-3
num_epochs = 50

trainloader, testloader = cifar10_loaders(image_size, batch_size)

Files already downloaded and verified
Files already downloaded and verified


### DeCAtt & MixedResolutionTokenizer

In [4]:
num_heads = 3
embed_dim = 64 * num_heads

model = MixedResViT(image_size, min_patch_size, max_patch_size, quadtree_num_patches, in_channels, 
                    embed_dim, num_heads, mlp_dim, num_layers, num_classes, dropout)                                                                                          
model = torch.nn.DataParallel(model.cuda(), device_ids=device_ids).cuda()

criterion = DecattLoss(num_heads)
optimizer1 = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)
optimizer2 = torch.optim.AdamW(model.module.transformer1.parameters(), lr=lr, weight_decay=weight_decay)
    
train("vit_mrt_cifar10", model, criterion, optimizer1, num_epochs, trainloader, testloader, optimizer2=optimizer2)

100%|██████████| 50/50 [1:49:51<00:00, 131.84s/it, Epoch=50, Train Accuracy=86.6, Training Loss=50.2, Validation Accuracy=75.9]

Best Validation Accuracy: 76.700%
Time to Max Val Accuracy: 103.270 mins
